# Explicitly Split KITTI into Train and Val

The KITTI dataset comes with a `trainval` folder of data. This small example notebook shows you how to use the `KittiObjectDataset` object to split the train and val splits into their own respective folders. It is not necessary to run this example to do anything of interest with KITTI, but it can help understand how the data are organized.

In [ ]:
import avstack
import avapi
import os
import shutil
from tqdm import tqdm

In [ ]:
KITTI_obj_base = "../data/KITTI/object"  # use your own path or add a symbolic link
KDM = avapi.kitti.KittiObjectDataset(KITTI_obj_base, 'training')  # the original "training" has both "train" and "val"

folders = ["train_only", "val_only"]  # these directories will be created
imset_names = ["train.txt", "val.txt"]  # these were downloaded with the ImageSet download
subfolders = ['velodyne', 'image_2', 'image_3', 'planes', 'label_2', 'calib']
exts       = ['.bin',     '.png',    '.png',    '.txt',   '.txt',    '.txt']

copy_data = False  # if true, actually copies. If false, makes a symbolic link for each file

for imset, fol in zip(imset_names, folders):
    os.makedirs(os.path.join(KITTI_obj_base, fol), exist_ok=True)
    for sub in subfolders:
        os.makedirs(os.path.join(fol, sub), exist_ok=True)

    # Read imagesets from downloaded files to know which files
    with open(os.path.join(KITTI_obj_base, 'ImageSets', imset), 'r') as f:
        idxs_this_split = [int(s.strip()) for s in f.readlines()]

    # Write image sets to the new name
    fol_split = fol.split('/')
    write_str = '\n'.join(['%06d'%i for i in idxs_this_split])
    with open('/' + os.path.join(*fol_split[:-1], 'ImageSets', fol_split[-1] + '.txt'), 'w') as f:
        f.write(write_str)
              
    # Copy or symlink the data
    for ext, sub in zip(exts, subfolders):
        print(f'{"Copying" if copy_data else "Making symolic links with"} {os.path.join(fol, sub)}')
        for idx in tqdm(idxs_this_split):
            src = os.path.join(KDM.split_path, sub, '%06d'%idx + ext)
            dest = os.path.join(fol, sub, '%06d'%idx + ext)
            if copy_data:
                shutil.copy2(src, dest)
            else:
                os.symlink(src, dest)

: 